**functools** 高阶函数和可调用对象上的操作

functools 模块应用于高阶函数，即参数或（和）返回值为其他函数的函数。通常来说，此模块公衡适用于所有可调用对象。

In [2]:
# @functools.cache(user_function)
'''
简单轻量级未绑定函数缓存。
返回值与 lru_cache(maxsize=None) 相同，创建一个查找函数参数的字典的简单包装器。 
因为它不需要移出旧值，所以比带有大小限制的 lru_cache() 更小更快。
'''
import functools
@functools.lru_cache
def factorial(n):
    return n * factorial(n-1) if n else 1

factorial(10)


# @functools.cached_property(func)
'''
将一个类方法转换为特征属性，一次性计算该特征属性的值，然后将其缓存为实例生命周期内的普通属性。 
类似于 property() 但增加了缓存功能。 
对于在其他情况下实际不可变的高计算资源消耗的实例特征属性来说该函数非常有用。
'''
import statistics
class DataSet:
    def __init__(self, sequence_of_numbers) -> None:
        self._data = tuple(sequence_of_numbers)

    @functools.cached_property
    def stdev(self):
        return statistics.stdev(self._data)
'''
cached_property() 的设定与 property() 有所不同。 
常规的 property 会阻止属性写入，除非定义了 setter。 
与之相反，cached_property 则允许写入。

cached_property 装饰器仅在执行查找且不存在同名属性时才会运行。
 当运行时，cached_property 会写入同名的属性。
  后续的属性读取和写入操作会优先于 cached_property 方法，其行为就像普通的属性一样。

缓存的值可通过删除该属性来清空。
 这允许 cached_property 方法再次运行。

注意，这个装饰器会影响 PEP 412 键共享字典的操作。 
这意味着相应的字典实例可能占用比通常时更多的空间。

而且，这个装饰器要求每个实例上的 __dict__ 是可变的映射。 
这意味着它将不适用于某些类型，例如元类（因为类型实例上的 __dict__ 属性是类命名空间的只读代理），
以及那些指定了 __slots__ 但未包括 __dict__ 作为所定义的空位之一的类（因为这样的类根本没有提供 __dict__ 属性）。

如果可变的映射不可用或者如果想要节省空间的键共享，
可以通过在 cache() 之上堆叠一个 property() 来实现类似 cached_property() 的效果:
'''

In [33]:
# functools.cmp_to_key(func)
from functools import cmp_to_key
import locale
'''
将(旧式的)比较函数转换为新式的 key function . 
在类似于 sorted() ， min() ， max() ， 
heapq.nlargest() ， heapq.nsmallest() ， itertools.groupby() 等函数的 key 参数中使用。
比较函数是可调用的，接受两个参数，比较这两个参数，例如：x,y 当x>y时返回1；等于时返回0；否则返回-1。
关键字函数也是可调用的，接受一个参数，同时返回一个可以用作排序关键字的值。
采用cmp_to_key()函数，可以接受两个参数，将两个参数做处理，例如：作和 作差等，转换成一个参数，即可应用于关键字函数。
'''
sorted('abc', key=cmp_to_key(locale.strcoll))
d = {'a':4, 'b': 2}
sorted(d.items(), key=lambda x: x[1])
# 当两者比较为正数交换位置，0 或 -1 不交换位置。
sorted([2, 1, 5, 3], key=cmp_to_key(lambda x, y: y - x))
sorted(d.items(), key=cmp_to_key(lambda x, y: x[1] - y[1]))
min([2, 3, 1, 3, 8], key=cmp_to_key(lambda x, y: y - x))
max([2, 3, 1, 3, 8], key=cmp_to_key(lambda x, y: x - y))

8

In [39]:
# @functools.total_ordering
'''
给定一个声明一个或多个全比较排序方法的类，这个类装饰器实现剩余的方法。
减轻了指定所有可能的全比较操作的工作。
此类必须包含以下方法之一：__lt__() 、__le__()、__gt__() 或 __ge__()。
另外，此类必须支持 __eq__() 方法。
'''
from functools import total_ordering
@total_ordering
class Student:
    def _is_valid_operand(self, other):
        return (hasattr(other, "lastname") and
                hasattr(other, "firstname"))
    def __eq__(self, other):
        if not self._is_valid_operand(other):
            return NotImplemented
        return ((self.lastname.lower(), self.firstname.lower()) ==
                (other.lastname.lower(), other.firstname.lower()))
    def __lt__(self, other):
        if not self._is_valid_operand(other):
            return NotImplemented
        return ((self.lastname.lower(), self.firstname.lower()) <
                (other.lastname.lower(), other.firstname.lower()))

a = Student()
a.lastname = 'b'
a.firstname= 'c'
b = Student()
b.lastname = 'a'
b.firstname= 'a'
a > b

True

In [45]:
from functools import total_ordering
@total_ordering
class Student:
    def _is_valid_operand(self, other):
        return (hasattr(other, "lastname") and
                hasattr(other, "firstname"))
    def __eq__(self, other):
        if not self._is_valid_operand(other):
            return NotImplemented
    def __lt__(self, other):
        if not self._is_valid_operand(other):
            return NotImplemented
        return ((self.lastname.upper(), self.firstname.lower()) <
                 (other.lastname.upper(), self.firstname.lower()))

a = Student()
a.lastname = 'b'
a.firstname= 'c'
b = Student()
b.lastname = 'c'
b.firstname = 'c'
a >= b

False

In [53]:
# functools.partial(func, /, *args, **keywards)
"""
返回一个新的 partial 对象，当被调用时其行为类似于 func 附带位置参数 args 和关键字参数
keywords 被调用。如果为调用提供了更多的参数，他们会被附加到args。如果提供了额外的关键字
参数，他们会扩展并重载 keywords。大致等价于：
"""
def partial(func, /, *args, **keywords):
    def newfunc(*fargs, **fkeywords):
        newkeywords = {**keywords, **fkeywords}
        return func(*args, *fargs, **newkeywords)
    newfunc.func = func
    newfunc.args = args
    newfunc.keywords = keywords
    return newfunc
"""
总的来说，就是将一个已有的函数或对象,固定某些参数，形成一个新的函数或对象。
"""
def test(name, age, *args, city=None,**kws):
    name = name
    age = age
    city = city
    yield tuple([name, age, city])

list(test(1, 2, city='g'))
from functools import partial
newtest = partial(test, city='beijing')
list(newtest(11, 22))

[(11, 22, 'beijing')]

In [59]:
# functools.partialmethod(func, /, *args, **keywords)
"""
返回一个新的 partialmethod 描述器，
其行为类似 partial 但它被设计用作方法定义而非直接用作可调用对象。

func 必须是一个 descriptor 或可调用对象（同属两者的对象例如普通函数会被当作描述器来处理）。

当 func 是一个描述器
（例如普通 Python 函数, classmethod(), staticmethod(), abstractmethod() 或其他 partialmethod 的实例）时, 
对 __get__ 的调用会被委托给底层的描述器，并会返回一个适当的 部分对象 作为结果。

当 func 是一个非描述器类可调用对象时，则会动态创建一个适当的绑定方法。
当用作方法时其行为类似普通 Python 函数：将会插入 self 参数作为第一个位置参数，
其位置甚至会处于提供给 partialmethod 构造器的 args 和 keywords 之前。
"""
from functools import partialmethod
class Cell:
    def __init__(self) -> None:
        self._alive = False
    @property
    def alive(self):
        return self._alive
    @alive.setter  
    def alive(self, _alive):
        self._alive = _alive

    def set_stat(self, state):
        self._alive = bool(state)

    set_alive = partialmethod(set_stat, True)
    set_dead = partialmethod(set_stat, False)
c = Cell()
c.alive
c.set_alive()
c.alive
# Cell.set_dead(c)
# c.alive

""" 
小结一下，就是在类里面使用，能替代一个属性的setter属性，
从个人来看，一般是在能设置属性较少的时候使用，两三个的时候。
"""

False

In [60]:
# functools.reduce(function, iterable[, intializer])
""" 
将两个参数的 function 从左至右积累地应用到 iterable 的条目，
以便将该可迭代对象缩减为单一的值。 
例如，reduce(lambda x, y: x+y, [1, 2, 3, 4, 5]) 是计算 ((((1+2)+3)+4)+5) 的值。
左边的参数 x 是积累值而右边的参数 y 则是来自 iterable 的更新值。 
如果存在可选项 initializer，它会被放在参与计算的可迭代对象的条目之前，
并在可迭代对象为空时作为默认值。 
如果没有给出 initializer 并且 iterable 仅包含一个条目，则将返回第一项。
"""
def reduce(function, iterable, initializer=None):
    it = iter(iterable)
    if initializer is None:
        value = next(it)
    else:
        value = initializer
    for element in it:
        value = function(value, element)
    return value

reduce(lambda x, y: x * y, [1, 2, 3, 4], initializer=2)

48

In [66]:
# functools.singledispatch
""" 
将一个函数转换为 单分派 generic function。
要定义一个泛型函数，用装饰器 @singledispatch 来装饰它。
当使用 @singledispatch 定义一个函数时，请注意调度发生在第一个参数的类型上:
"""
from functools import singledispatch
@singledispatch
def fun(arg, verbose=False):
    if verbose:
        print("Let me just say,", end=" ")
    print(arg)

fun([1, 2, 3])
""" 
要将重载的实现添加到函数中，请使用泛型函数的 register() 属性，它可以被用作装饰器。 
对于带有类型标注的函数，该装饰器将自动推断第一个参数的类型:
"""
@fun.register
def _(arg: int, verbose=False):
    if verbose:
        print("Strength in numbers, eh?", end=" ")
    print(arg)

_(1)

@fun.register
def _(arg: list, verbose=False):
    if verbose:
        print("Enumerate this:")
    for i, elem in enumerate(arg):
        print(i, elem)
_('abc')

[1, 2, 3]
1
0 a
1 b
2 c


In [71]:
from functools import wraps
def my_decorator(f):
    @wraps(f)
    def wrapper(*args, **kwds):
        print('Calling decorated function')
        return f(*args, **kwds)
    return wrapper

@my_decorator
def example():
    """Docstring"""
    print('Called example function')

example()
example.__name__

Calling decorated function
Called example function


'example'

## functools
### 1. cmp_to_key(func)
可以将cmp_to_key(func)传递给那些接受key，关键字参数的函数，sorted()，max()，min()等
其中，在func中返回值为正数就交换两者位置，其余情况不发生改变

### 2. total_ordering 装饰器
定义一个类，只需要实现__eq__() 和 __lt__() 、__le__()、__gt__() 或 __ge__()其中一个
就可以实现全比较类。

### 3. partial(func, /, *args, **keywords)
能够固定某个参数，形成新的函数

### 4. reduce(function, iterable[, initializer])
将iterable缩减为一个元素

### 5. @functools.wraps() 可以保护装饰器内部函数名不发生改变
from functools import wraps
def my_decorator(f):
    @wraps(f)
    def wrapper(*args, **kwds):
        print('Calling decorated function')
        return f(*args, **kwds)
    return wrapper

@my_decorator
def example():
    """Docstring"""
    print('Called example function')

example()


example.__name__

example.__doc__